# Fruit Classification
### Transfer learning of VGG16 model to classify fresh and rotten fruits from Kaggle dataset

### Load and prepare model for data

In [1]:
# Initial import of Keras for use throughout project
from tensorflow import keras

In [3]:
# Load base model (VGG16)
base = keras.applications.VGG16(
    weights = "imagenet", # Use imagenet weights to match
    input_shape = (224, 224, 3), # Specify input shape to match dataset (224 x 224 pixels, 3 color channels)
    include_top = False # Remove output layer
)

58889256/58889256 [==============================] - 1s 0us/step


In [4]:
# Verify removal of output layer
base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [5]:
# Freeze base layers to retain learning
base.trainable = False

In [6]:
# Specify inputs with correct shape
inputs = keras.Input(shape = (224, 224, 3))
x = base(inputs, training = False)

# Add pooling layer
x = keras.layers.GlobalAveragePooling2D()(x)

# Add dense output layer
outputs = keras.layers.Dense(6, activation = "softmax")(x)

# Combine to create new model
model = keras.Model(inputs, outputs)

In [7]:
# Verify model layers
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 global_average_pooling2d (G  (None, 512)              0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 6)                 3078      
                                                                 
Total params: 14,717,766
Trainable params: 3,078
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
# Compile the model
model.compile(
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = [keras.metrics.Accuracy()]
)

### Data Preparation

In [3]:
# Augment data to increase model accuracy when trained
from keras.preprocessing.image import ImageDataGenerator

# Create data generator for training dataset
datagen_train = ImageDataGenerator(
    samplewise_center = True, # Set sample mean to 0
    rotation_range = 10, # Randomly rotate images 10 degrees
    zoom_range = 0.1, # Randomly zoom image
    width_shift_range = 0.1, # Randomly shift image horizontally
    height_shift_range = 0.1, # Randomly shift image vertically
    horizontal_flip = True, # Allow horizontal flipping
    vertical_flip = False # Don't allow vertical flipping
)

# No augmenting needed for testing data
datagen_test = ImageDataGenerator(
    samplewise_center = True # Set sample mean to 0
)